In [1]:
from datetime import datetime
import os
import time
import requests
import json

import pickle
from pathlib import Path
import traceback

In [2]:
# call API to retrieve comments of a game with a game id

# game_steamid = 1716740          # starfield
# game_name = 'starfield'

# game_steamid = 2138330          # cyberpunk2077 phantom liberty
# game_name = 'cyberpunk2077_phantom_liberty'

# game_steamid = 1091500          # cyberpunk2077
# game_name = 'cyberpunk2077'

# game_steamid = 1118010
# game_name = 'monster_hunter_world_iceborne'

# game_steamid = 582010
# game_name = 'monster_hunter_world'

# game_steamid = 730
# game_name = 'counter-strike_2'

# game_steamid = 570
# game_name = "dota2"

# game_steamid = 2138330
# game_name = 'cyberpunk2077_phantom_liberty'

# game_steamid = 1814540
# game_name = 'cosmos_conquer'

# game_steamid = 1814520
# game_name = 'find_the_cat'

# game_steamid = 1814480
# game_name = 'fourplay_chess'

# game_steamid = 1757970
# game_name = 'oasis_dark_forest'

# game_steamid = 1607720
# game_name = 'rimebeard'

# game_steamid = 1814460
# game_name = 'the_brave_vs_dragon'

# game_steamid = 1814510
# game_name = 'yavi'

game_steamid_name_list = [
    (1716740, 'starfield'),
    (2138330, 'cyberpunk2077_phantom_liberty'),
    (1091500, 'cyberpunk2077'),
    (1118010, 'monster_hunter_world_iceborne'),
    (582010, 'monster_hunter_world'),
    (730, 'counter-strike_2'),
    (570, 'dota2'),
    (2138330, 'cyberpunk2077_phantom_liberty'),
    (1814540, 'cosmos_conquer'),
    (1814520, 'find_the_cat'),
    (1814480, 'fourplay_chess'),
    (1757970, 'oasis_dark_forest'),
    (1607720, 'rimebeard'),
    (1814460, 'the_brave_vs_dragon'),
    (1814510, 'yavi')
]

params = {
    'json':1,
    'cursor':'*',
    'num_per_page':100,
    'language':'english',       # only english reviews
    'filter':'recent',
    'review_type':'all',
    'purchase_type':'all'

}

# api doc: https://partner.steamgames.com/doc/store/getreviews
# reviews_req = requests.get(f"https://store.steampowered.com/appreviews/{game_steamid}", params=params)

In [7]:
# load the reviews from folder

reviews_reqs_list = []

# get existing folder and retrieve the cursor object (?)

# load the latest file

for (game_steamid, game_name) in game_steamid_name_list:
    game_folder = Path(f'./{game_name}')
    if game_folder.exists():
        try:
            latest_file_path = sorted(game_folder.glob(f'steam_reviews_{game_steamid}_2024*.pkl'))[-1]
            with open(latest_file_path, 'rb') as f:
                reviews_reqs = pickle.load(f)           # retrieve the list of reviews
                # cursor_str = reviews_reqs[-1]['cursor']
                print('Loaded:', latest_file_path)
                # print('Cursor:', cursor_str)

                reviews_reqs_list.append(reviews_reqs)
        except IndexError as e:
            print('Error loading the latest file:', e)
            traceback.print_exc()

Loaded: starfield/steam_reviews_1716740_20240210191441.pkl
Loaded: cyberpunk2077_phantom_liberty/steam_reviews_2138330_20240311161752.pkl
Loaded: cyberpunk2077/steam_reviews_1091500_20240210230848.pkl
Loaded: monster_hunter_world_iceborne/steam_reviews_1118010_20240224170528.pkl
Loaded: monster_hunter_world/steam_reviews_582010_20240226135807.pkl
Loaded: counter-strike_2/steam_reviews_730_20240226235729.pkl
Loaded: dota2/steam_reviews_570_20240311164236.pkl
Loaded: cyberpunk2077_phantom_liberty/steam_reviews_2138330_20240311161752.pkl
Loaded: cosmos_conquer/steam_reviews_1814540_20240311164748.pkl
Loaded: find_the_cat/steam_reviews_1814520_20240311164657.pkl
Loaded: fourplay_chess/steam_reviews_1814480_20240311164511.pkl
Loaded: oasis_dark_forest/steam_reviews_1757970_20240311164846.pkl
Loaded: rimebeard/steam_reviews_1607720_20240311164551.pkl
Loaded: the_brave_vs_dragon/steam_reviews_1814460_20240311164426.pkl
Loaded: yavi/steam_reviews_1814510_20240311164621.pkl


In [8]:
for reviews_reqs in reviews_reqs_list:
    print('Number of reviews:', len(reviews_reqs))

Number of reviews: 1088
Number of reviews: 67
Number of reviews: 5600
Number of reviews: 70
Number of reviews: 1431
Number of reviews: 2550
Number of reviews: 607
Number of reviews: 67
Number of reviews: 1
Number of reviews: 1
Number of reviews: 1
Number of reviews: 1
Number of reviews: 1
Number of reviews: 1
Number of reviews: 1


In [15]:
len(reviews_reqs)

1088

In [9]:
number_of_reviews_list = []


for reviews_reqs in reviews_reqs_list:

    review_list = []

    # datetime_until = datetime(2023, 1, 1, 0, 0, 0)

    for reviews_req in reviews_reqs:
        review_list.extend(reviews_req['reviews'])

    # filter by unique 'recommendationid'
    unique_review_list = []
    unique_review_ids = set()

    for review in review_list:
        if review['recommendationid'] in unique_review_ids:
            continue

        # if review['timestamp_created'] < datetime_until.timestamp():
        #     continue

        unique_review_list.append(review)
        unique_review_ids.add(review['recommendationid'])

    number_of_reviews_list.append(len(unique_review_list))

In [11]:
for (game_steamid, game_name), number_of_reviews in zip(game_steamid_name_list, number_of_reviews_list):
    print(f'Number of unique reviews in {game_name}:', number_of_reviews)

Number of unique reviews in starfield: 96734
Number of unique reviews in cyberpunk2077_phantom_liberty: 6673
Number of unique reviews in cyberpunk2077: 299934
Number of unique reviews in monster_hunter_world_iceborne: 6913
Number of unique reviews in monster_hunter_world: 143026
Number of unique reviews in counter-strike_2: 254993
Number of unique reviews in dota2: 60697
Number of unique reviews in cyberpunk2077_phantom_liberty: 6673
Number of unique reviews in cosmos_conquer: 2
Number of unique reviews in find_the_cat: 2
Number of unique reviews in fourplay_chess: 1
Number of unique reviews in oasis_dark_forest: 4
Number of unique reviews in rimebeard: 4
Number of unique reviews in the_brave_vs_dragon: 2
Number of unique reviews in yavi: 15


Very similar to the time I read the number of reviews: 96741, at 20240211:000030

In [44]:
# save the unique reviews

# with open(f'./{game_name}/steam_reviews_{game_steamid}_unique.pkl', 'wb') as f:
#     pickle.dump(unique_review_list, f)

The number excludes off-topic reviews (??)

---